In [15]:
import pandas as pd

# Read the CSV file into a pandas DataFrame (replace 'data/AE9200_prob01_profiles.csv' with your file path)
df = pd.read_csv('data/AB0208_prob01_profiles.csv')

# Separate the name and address columns
df[['first name', 'last name']] = df['name'].str.split(' ', 1, expand=True)

# Split the 'address' column into multiple columns
address_columns = df['address'].str.split('\n', expand=True)

# Check the number of elements after splitting
num_address_columns = address_columns.shape[1]

# Adjust column assignment based on the number of elements
if num_address_columns >= 3:
    df[['street address', 'state', 'postal code']] = address_columns.iloc[:, :3]
elif num_address_columns == 2:
    df[['street address', 'state']] = address_columns.iloc[:, :2]
    df['postal code'] = None  # Add a placeholder for missing postal code
else:
    df['street address'] = address_columns.iloc[:, 0]
    df['state'] = None  # Add a placeholder for missing state
    df['postal code'] = None  # Add a placeholder for missing postal code

# Keep only the relevant columns
df = df[['ssn', 'username', 'sex', 'mail', 'birthdate', 'first name', 'last name', 'street address', 'state', 'postal code']]

# Filter and sort the entries as specified
filtered_entries = df[df['last name'].str.startswith('J')].sort_values(by=['sex', 'state', 'birthdate'])

# Print the entries in three different ways
print("Sorted by sex (ladies first), state, age (youngest first):")
print(filtered_entries)

print("\nSorted by state (alphabetically), sex, age (youngest first):")
print(filtered_entries.sort_values(by=['state', 'sex', 'birthdate']))

print("\nSorted by age (youngest first), sex, state:")
print(filtered_entries.sort_values(by=['birthdate', 'sex', 'state']))


Sorted by sex (ladies first), state, age (youngest first):
               ssn         username sex                          mail  \
271    ZZ 877887 T        francis16   F             derek82@yahoo.com   
141    ZZ 066568 T         joanne15   F            zclark@outlook.com   
408    ZZ 899734 T    teresabarrett   F   oneillstephen@hotmail.co.uk   
319  ZZ 56 97 44 T   rosemarytaylor   F   alexandermorris@hotmail.com   
496      ZZ326164T      jonesrachel   F        stuart59@hotmail.co.uk   
251  ZZ 91 03 89 T    chapmanduncan   F         nobledean@hotmail.com   
142      ZZ465559T     edwardsbrett   F           qharris@hotmail.com   
119      ZZ857865T       elliewells   F           ttaylor@outlook.com   
289      ZZ498409T       clarkebeth   F      reecewhittaker@gmail.com   
293      ZZ759076T          gburton   F     russellnewman@yahoo.co.uk   
52     ZZ 779963 T         adrian00   F           mrogers@yahoo.co.uk   
46     ZZ 909920 T  clementsharriet   F       jeanroberts@outlook

In [23]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
weather_df = pd.read_csv('data/AB0208_prob02_weather.csv')

# Convert the 'Time' column to datetime format
weather_df['Time'] = pd.to_datetime(weather_df['Time'])

# Filter relevant columns and rows
filtered_df = weather_df[weather_df['ParameterName'].isin(['tmax', 'tmin'])]

# Pivot the DataFrame to have 'tmax' and 'tmin' as separate columns
pivot_df = filtered_df.pivot(index='Time', columns='ParameterName', values='ParameterValue')

# Calculate the average of (tmax + tmin)/2 using .loc to avoid SettingWithCopyWarning
pivot_df.loc[:, 'avg_temp'] = (pivot_df['tmax'] + pivot_df['tmin']) / 2

# Calculate one standard deviation away from the total average
std_dev = pivot_df['avg_temp'].std()
one_std_dev_away = (pivot_df['avg_temp'] >= (pivot_df['avg_temp'].mean() - std_dev)) & (pivot_df['avg_temp'] <= (pivot_df['avg_temp'].mean() + std_dev))

# Calculate the percentage of observations within one standard deviation
percentage_within_one_std_dev = (one_std_dev_away.sum() / len(pivot_df)) * 100

print(f"a) Percentage of observations within one standard deviation: {percentage_within_one_std_dev:.2f}%\n")

# Find the top-5 timestamps for the difference between tmax and tmin
pivot_df['temp_diff'] = pivot_df['tmax'] - pivot_df['tmin']
top5_temp_diff = pivot_df.nlargest(5, 'temp_diff')

# Print information for the top-5 rows
print("b) Top-5 timestamps for the difference between tmax and tmin:")
for index, row in top5_temp_diff.iterrows():
    print(f"Timestamp: {index}, Max Temperature: {row['tmax']}, Min Temperature: {row['tmin']}, Temperature Difference: {row['temp_diff']}")


a) Percentage of observations within one standard deviation: 3.19%

b) Top-5 timestamps for the difference between tmax and tmin:
Timestamp: 2022-01-11 00:00:00+00:00, Max Temperature: -4.9, Min Temperature: -17.6, Temperature Difference: 12.700000000000001
Timestamp: 2022-01-08 00:00:00+00:00, Max Temperature: -2.6, Min Temperature: -13.4, Temperature Difference: 10.8
Timestamp: 2022-01-23 00:00:00+00:00, Max Temperature: 1.8, Min Temperature: -8.9, Temperature Difference: 10.700000000000001
Timestamp: 2022-01-10 00:00:00+00:00, Max Temperature: -5.0, Min Temperature: -13.9, Temperature Difference: 8.9
Timestamp: 2022-01-06 00:00:00+00:00, Max Temperature: 0.1, Min Temperature: -8.4, Temperature Difference: 8.5


In [25]:
import pandas as pd

# Load the data
epl_df = pd.read_csv('./data/AB0208_prob03_epl.csv')

# Initialize the table with unique team names as the index
table_df = pd.DataFrame(index=epl_df['homeTeam'].unique())

# Initialize columns with zeros
table_df['games'] = 0
table_df['wins'] = 0
table_df['draws'] = 0
table_df['defeats'] = 0
table_df['goals_for'] = 0
table_df['goals_against'] = 0
table_df['points'] = 0

# Calculate statistics
for _, row in epl_df.iterrows():
    home_team = row['homeTeam']
    away_team = row['awayTeam']
    full_time = eval(row['fullTime'])  # Using eval to convert the string representation of the dictionary to a dictionary
    goals1 = full_time['homeTeam']
    goals2 = full_time['awayTeam']

    # Update games played
    table_df.at[home_team, 'games'] += 1
    table_df.at[away_team, 'games'] += 1

    # Update goals for and against
    table_df.at[home_team, 'goals_for'] += goals1
    table_df.at[home_team, 'goals_against'] += goals2
    table_df.at[away_team, 'goals_for'] += goals2
    table_df.at[away_team, 'goals_against'] += goals1

    # Update wins, draws, and defeats
    if goals1 > goals2:
        table_df.at[home_team, 'wins'] += 1
        table_df.at[away_team, 'defeats'] += 1
    elif goals1 < goals2:
        table_df.at[home_team, 'defeats'] += 1
        table_df.at[away_team, 'wins'] += 1
    else:
        table_df.at[home_team, 'draws'] += 1
        table_df.at[away_team, 'draws'] += 1

# Calculate goal difference
table_df['goal_difference'] = table_df['goals_for'] - table_df['goals_against']

# Calculate points
table_df['points'] = 3 * table_df['wins'] + table_df['draws']

# Sort the table
table_df = table_df.sort_values(by=['points', 'goal_difference', 'goals_for'], ascending=False)

# Display the table
print(table_df)


                            games  wins  draws  defeats  goals_for  \
Manchester City FC             38    29      6        3         99   
Liverpool FC                   38    28      8        2         94   
Chelsea FC                     38    21     11        6         76   
Tottenham Hotspur FC           38    22      5       11         69   
Arsenal FC                     38    22      3       13         61   
Manchester United FC           38    16     10       12         57   
West Ham United FC             38    16      8       14         60   
Leicester City FC              38    14     10       14         62   
Brighton & Hove Albion FC      38    12     15       11         42   
Wolverhampton Wanderers FC     38    15      6       17         38   
Newcastle United FC            38    13     10       15         44   
Crystal Palace FC              38    11     15       12         50   
Brentford FC                   38    13      7       18         48   
Aston Villa FC      

In [31]:
import pandas as pd

# Assuming 'Time' is already in datetime format
# If not, you can convert it using: df['Time'] = pd.to_datetime(df['Time'])

# Read the sample data into a DataFrame
df = pd.read_csv('./data/AB0208_prob04_weather.csv')

# Display all column names
print(df.columns)

# Choose the correct column name for snow depth
snow_depth_column_name = 'Snow depth (cm)'

# Replace -1 with 0 for snow depth
df[snow_depth_column_name] = df[snow_depth_column_name].replace(-1, 0)

# Fill missing values with the previous valid value
df[snow_depth_column_name] = df[snow_depth_column_name].fillna(method='ffill')

# Define winters (December to February)
winter_months = [12, 1, 2]
df['Winter'] = df['Year'].astype(str) + '-' + (df['Month'] // 12 + df['Month'] % 12).astype(str)

# Calculate snow sum for each winter
winter_snow_sum = df.groupby('Winter')[snow_depth_column_name].sum()

# Calculate rank, count of positive snow depth days, and max snow depth for each winter
winter_stats = pd.DataFrame({
    'Snow sum': winter_snow_sum,
    'Rank': winter_snow_sum.rank(ascending=False).astype(int),
    'Count': df[df[snow_depth_column_name] > 0].groupby('Winter')[snow_depth_column_name].count(),
    'Max': df.groupby('Winter')[snow_depth_column_name].max()
})

# Display the result
print(winter_stats)


Index(['Year', 'Month', 'Day', 'Time', 'TimeZone', 'Precipitation (mm)',
       'Snow depth (cm)', 'Air temperature', 'Max air temperature',
       'Min air temperature'],
      dtype='object')
         Snow sum  Rank  Count   Max
Winter                              
1959-1     2180.0    16   62.0  64.0
1959-10      98.0   280    9.0  20.0
1959-11     144.0   261   12.0  15.0
1959-2     1502.0    73   28.0  63.0
1959-3     1439.0    84   31.0  55.0
...           ...   ...    ...   ...
2020-6        0.0   527    NaN   0.0
2020-7        0.0   527    NaN   0.0
2020-8        0.0   527    NaN   0.0
2020-9        0.0   527    NaN   0.0
2021-1      469.0   210   24.0  33.0

[683 rows x 4 columns]


In [97]:
import json
import pandas as pd

with open('test.json', 'r') as json_file:
    data = json.load(json_file)

years = []
background_countries = []
live_births = []

for item in data['data']:
    # Extract the 'key' and 'values' from the item
    year, country_code = item['key']
    values = item.get('values', [])

    if values and isinstance(values[0], dict):
        # Check if the first element of 'values' is a dictionary
        first_value = values[0]

        # Extract 'columns' from the first element
        columns = first_value.get('columns', [])

        if columns and isinstance(columns, list):
            # Check if 'columns' is a list
            live_birth = columns[0]
        elif columns and isinstance(columns, dict):
            # Check if 'columns' is a dictionary
            # Modify this line based on the actual structure of 'columns'
            live_birth = columns.get('your_key', None)
        else:
            print("Unexpected type for 'columns'.")
            live_birth = None
   # else:
    #    print("No 'values' or empty list.")

    years.append(year)
    background_countries.append(country_code)
    live_births.append(live_birth)

df = pd.DataFrame({
    'Year': years,
    'Background Country Code': background_countries,
    'Live Births': pd.to_numeric(live_births, errors='coerce')
})

df


,Year,Background Country Code,Live Births
0,2015,0,NaN
1,2015,01,NaN
2,2015,011,NaN
3,2015,0111,NaN
4,2015,01111,NaN
...,...,...,...
6610,2023,1270,NaN
6611,2023,12701,NaN
6612,2023,127011,NaN
6613,2023,12704,NaN
